# StyleGAN3

In [ ]:
!pip3 install tensorflow
!git clone https://github.com/NVlabs/stylegan3.git
%cd stylegan3

In [ ]:
# Load Pre-trained StyleGAN3 Model using the FFHQ dataset for generating faces

import dnnlib
import numpy as np
import PIL.Image
from tqdm import tqdm
import legacy
import torch

# Ensure PyTorch is using the CPU
device = torch.device('cpu')

# Load pre-trained network
url = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl'
with dnnlib.util.open_url(url) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) 

# Generate images
for i in range(5): 
    # Generate latent vector
    z = np.random.randn(1, G.z_dim)
    z = torch.from_numpy(z).to(device)  # Convert to torch tensor and move to CPU

    # Generate image
    img = G(z, None)  # Generate images from latent vectors
    img = (img * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    img = img.permute(0, 2, 3, 1)[0].cpu().numpy()  # Convert to numpy array

    # Save image
    PIL.Image.fromarray(img, 'RGB').save(f'fake_face_sg3_{i}.png')

# Stable Diffusion

In [ ]:
!pip3 install diffusers transformers scipy

In [ ]:
from diffusers import StableDiffusionPipeline

# Load the model, requires Hugging Face token
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", 
                                               use_auth_token=True)
pipe = pipe.to("cuda")

In [ ]:
prompt = "A passport photo" 
with torch.no_grad():
    for i in range(5): # generate 5 images
        image = pipe(prompt).images[0]
        image.save(f"fake_face_sd_{i}.png")